In [1]:
import xlwings as xw
import pandas as pd

from Options_Math_Helpers import *
from Options_Math_Algebra import *
from Options_Math_Black_Scholes import *

oma  = OptionsMathAlgebra()
ombs = OptionsMathBlackScholes()

In [2]:
# Open workbook
wb = xw.Book('deribit.xlsx')

# Select sheet by name
sheet = wb.sheets['combo_with_time']

# Read entire used range into a DataFrame
raw_df = sheet.used_range.options(pd.DataFrame, header=True, index=False).value

In [3]:
cols = ['kind', 
        'instrument_name', 
        'option_type', 
        'strike', 
        'expiration_time_nyc', 
        'years_to_expiry', 
        'underlying_price',
        'mark_price', 
        'best_bid_price', 
        'best_ask_price',
        'mark_iv', 
        'bid_iv', 
        'ask_iv', 
        'greeks_vega'
       ]

df = raw_df.loc[:, [c for c in cols if c in raw_df.columns]]
df = df.loc[df['kind'] == 'option']
df = df.drop(columns=['kind'])

print('done')

done


In [4]:
opt_type = df['option_type']
fwd      = df['underlying_price']
strike   = df['strike']
time     = df['years_to_expiry'] #* 365/365.25

df['mark_dollar']     = df['underlying_price'] * df['mark_price']
df['best_bid_dollar'] = df['underlying_price'] * df['best_bid_price']  
df['best_ask_dollar'] = df['underlying_price'] * df['best_ask_price'] 

In [5]:
df['intrinsic'] = oma.intrinsic_value(opt_type=opt_type, spot=fwd, strike_pv=strike) # both are fv
df['time_val_bid'] = df['best_bid_dollar'] - df['intrinsic']
df['time_val_ask'] = df['best_ask_dollar'] - df['intrinsic']

In [6]:
call_df = df[df['option_type'] == 'call']
put_df  = df[df['option_type'] == 'put']

for i in call_df.index:
    for bid_ask in ['bid', 'ask']:
        col_name1 = 'time_val_' + bid_ask
        call_tv = call_df.loc[i, col_name1]

        call_k = call_df.loc[i, 'strike']
        call_exp = call_df.loc[i, 'expiration_time_nyc']

        # Correct boolean mask
        mask_put = (put_df['strike'] == call_k) & (put_df['expiration_time_nyc'] == call_exp)
        put_tv = put_df.loc[mask_put, col_name1].iloc[0]
        
        if bid_ask == 'bid':
            best_tv = max(call_tv, put_tv)
        else:
            best_tv = min(call_tv, put_tv)

        # Assign back to df
        col_name2 = 'best_tv_strike_' + bid_ask
        mask_stats = (df['strike'] == call_k) & (df['expiration_time_nyc'] == call_exp)
        df.loc[mask_stats, col_name2] = best_tv

df['best_tv_strike_bid'] = np.maximum(0, df['best_tv_strike_bid'])

In [7]:
df = df.sort_values(
    by=['expiration_time_nyc', 'strike', 'option_type'])

unique_expirations = df['expiration_time_nyc'].unique()

In [8]:
hi_strike_mask = df['strike'] > df['underlying_price']
lo_strike_mask = ~hi_strike_mask

for exp in unique_expirations:
    exp_mask = df['expiration_time_nyc'] == exp

    # bids first
    combo_mask = lo_strike_mask & exp_mask
    s = df.loc[combo_mask, 'best_tv_strike_bid']
    df.loc[combo_mask, 'enriched_bid_tv'] = s.cummax()

    combo_mask = hi_strike_mask & exp_mask
    s = df.loc[combo_mask, 'best_tv_strike_bid']
    df.loc[combo_mask, 'enriched_bid_tv'] = s.iloc[::-1].cummax().iloc[::-1]
    
    # asks second
    combo_mask = hi_strike_mask & exp_mask
    s = df.loc[combo_mask, 'best_tv_strike_ask']
    df.loc[combo_mask, 'enriched_ask_tv'] = s.cummin()
    
    combo_mask = lo_strike_mask & exp_mask
    s = df.loc[combo_mask, 'best_tv_strike_ask']
    df.loc[combo_mask, 'enriched_ask_tv'] = s.iloc[::-1].cummin().iloc[::-1]

In [9]:
df['enriched_bid_price'] = df['intrinsic'] + df['enriched_bid_tv']
df['enriched_ask_price'] = df['intrinsic'] + df['enriched_ask_tv']
df['enriched_mid_price'] = (df['enriched_bid_price'] + df['enriched_ask_price']) / 2

In [10]:
df['enriched_bid_iv'] = ombs.bs_vol_solver_newton(opt_value=df['enriched_bid_price'], opt_type=opt_type, fwd=fwd, strike=strike, time=time)
df['enriched_bid_iv'] = df['enriched_bid_iv'] * 100

df['enriched_ask_iv'] = ombs.bs_vol_solver_newton(opt_value=df['enriched_ask_price'], opt_type=opt_type, fwd=fwd, strike=strike, time=time)
df['enriched_ask_iv'] = df['enriched_ask_iv'] * 100

df['enriched_mid_iv'] = (df['enriched_bid_iv'] + df['enriched_ask_iv']) / 2

C:\Users\micha\OneDrive\Deribit\Options_Math_Black_Scholes.py:387: RuntimeWarning: divide by zero encountered in divide
  safe_step = np.where(np.abs(vega) > 1e-8, 0.01 * diff / vega, 0.0)
C:\Users\micha\OneDrive\Deribit\Options_Math_Black_Scholes.py:387: RuntimeWarning: divide by zero encountered in divide
  safe_step = np.where(np.abs(vega) > 1e-8, 0.01 * diff / vega, 0.0)


In [11]:
sheet = wb.sheets('enriched')
sheet.range('a1').options(index = False, header=1).value = df